# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 6


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 935
po2 935
albumin 558
hba1c 444
lymphocyte 585
height 415
urine-ketone 343
crp 57


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,13616762,20066510,31257435,True,NaN,64,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,NaN,3,0.0,0,NaN,130.0
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,NaN,5,0.0,0,NaN,NaN
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,3,0.0,1,NaN,NaN
0,18135181,20173681,30217596,True,NaN,54,NaN,NaN,NaN,NaN,...,NaN,24,NaN,NaN,NaN,1,0.0,0,NaN,103.5
0,12851923,20235113,35081736,False,NaN,84,NaN,NaN,NaN,NaN,...,NaN,42,NaN,NaN,NaN,5,0.0,1,NaN,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,11084536,29816115,31328473,False,NaN,25,NaN,NaN,NaN,NaN,...,NaN,17,NaN,NaN,NaN,1,0.0,0,NaN,NaN
0,11166681,29877040,38192611,True,NaN,69,NaN,NaN,NaN,NaN,...,NaN,29,NaN,NaN,NaN,4,0.0,0,NaN,NaN
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,NaN,3,0.0,0,NaN,47.4
0,18624988,29915993,31559071,True,NaN,82,NaN,NaN,NaN,NaN,...,NaN,63,NaN,NaN,NaN,11,0.0,0,NaN,120.3


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

7879694

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

7609128

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-20 16:09:45.989008: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-20 16:09:46.505515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-20 16:09:46.907894: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 16:09:46.911394: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an 

 1/25 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - AUC: 0.3583 - accuracy: 0.3438 - loss: 0.7094 - precision: 0.4545 - recall: 0.2500

20/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.4881 - accuracy: 0.4491 - loss: 0.6974 - precision: 0.4655 - recall: 0.2307

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.5119 - accuracy: 0.4722 - loss: 0.6946 - precision: 0.4961 - recall: 0.2574 - val_AUC: 0.7015 - val_accuracy: 0.6598 - val_loss: 0.6650 - val_precision: 0.6782 - val_recall: 0.6082


...

20/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9066 - accuracy: 0.8278 - loss: 0.3949 - precision: 0.8324 - recall: 0.8166 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9035 - accuracy: 0.8236 - loss: 0.3989 - precision: 0.8326 - recall: 0.8093 - val_AUC: 0.7366 - val_accuracy: 0.6392 - val_loss: 0.6672 - val_precision: 0.6226 - val_recall: 0.6875


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - AUC: 0.5060 - accuracy: 0.4375 - loss: 0.7552 - precision: 0.5000 - recall: 0.4444

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6873 - accuracy: 0.6258 - loss: 0.6449 - precision: 0.6645 - recall: 0.5912 


Loses: [0.5738053321838379, 0.57042396068573, 0.5472708344459534, 0.6205055117607117, 0.6287672519683838] 0.5881545782089234 0.03126572671435283
AUCs: [0.7739290595054626, 0.7784382104873657, 0.8146477937698364, 0.710006833076477, 0.7089130878448486] 0.7571869969367981 0.04144874442010797
Accuracies: [0.7066115736961365, 0.71074378490448, 0.7479338645935059, 0.6652892827987671, 0.6431535482406616] 0.6947464108467102 0.03675635772754058
Precisions: [0.703125, 0.7086614370346069, 0.7540983557701111, 0.6589147448539734, 0.6636363863945007] 0.6976871848106384 0.03462653094925168
Recals: [0.7317073345184326, 0.7317073345184326, 0.7479674816131592, 0.6967213153839111, 0.5983606576919556] 0.7012928247451782 0.054119383062542804


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - AUC: 0.4494 - accuracy: 0.4375 - loss: 0.7112 - precision: 0.4000 - recall: 0.7692

20/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5609 - accuracy: 0.5343 - loss: 0.6888 - precision: 0.5254 - recall: 0.7905

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.5793 - accuracy: 0.5472 - loss: 0.6860 - precision: 0.5363 - recall: 0.7854 - val_AUC: 0.7265 - val_accuracy: 0.6788 - val_loss: 0.6526 - val_precision: 0.6915 - val_recall: 0.6633


Epoch 2/50


...

19/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9380 - accuracy: 0.8666 - loss: 0.3202 - precision: 0.9018 - recall: 0.8178 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9377 - accuracy: 0.8648 - loss: 0.3199 - precision: 0.9019 - recall: 0.8163 - val_AUC: 0.7110 - val_accuracy: 0.6425 - val_loss: 0.8363 - val_precision: 0.6593 - val_recall: 0.6122


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - AUC: 0.5298 - accuracy: 0.4062 - loss: 0.8095 - precision: 0.4615 - recall: 0.3333

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6838 - accuracy: 0.6093 - loss: 0.6619 - precision: 0.6893 - recall: 0.4799 


Loses: [0.5818882584571838, 0.5649633407592773, 0.536788821220398, 0.6153150200843811, 0.6355939507484436] 0.5869098782539368 0.03520221165951065
AUCs: [0.765389084815979, 0.7777891159057617, 0.8166974782943726, 0.7261953353881836, 0.7127703428268433] 0.759768271446228 0.03723768601941165
Accuracies: [0.6859503984451294, 0.7190082669258118, 0.7438016533851624, 0.6735537052154541, 0.6556016802787781] 0.6955831408500671 0.03178164386181008
Precisions: [0.7117117047309875, 0.7272727489471436, 0.7652173638343811, 0.6776859760284424, 0.7349397540092468] 0.7233655095100403 0.02871667916405659
Recals: [0.642276406288147, 0.7154471278190613, 0.7154471278190613, 0.6721311211585999, 0.5] 0.6490603566169739 0.07951311697406306
